### **Organize publicly available data about startups turned unicorns.**
#### In venture capital, a unicorn company is a privately-held startup company that is valued at more than $1 billion. <br> It’s a term that’s become common in the investing community.

To load and visualize the data, we can use the Pandas and Bokeh library

In [42]:
%pip install pandas
%pip install bokeh

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'D:\Personal\PersonalUseSoftware\Python\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'D:\Personal\PersonalUseSoftware\Python\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'D:\Personal\PersonalUseSoftware\Python\python.exe -m pip install --upgrade pip' command.


In [43]:
import pandas as pd
from bokeh.plotting import figure, show
from bokeh.io import output_notebook

# Load BokehJS plugin
output_notebook()

Loading BokehJS ...

Read the cleaned dataset present in '../dataset/' directory

In [51]:
# Read the dataset using read_csv function
df_all = pd.read_csv('../dataset/unicorn.csv')

Transform and organize the data, to make them ready for visualization
1. $ sign scrapped from Valuation ($B) column values
2. Valuation ($B) column values converted from object to float64
3. Date Joined column values converted from object to python legible datetime64

In [52]:
# Strip the $ sign from the valuation column
df_all['Valuation ($B)'] = df_all['Valuation ($B)'].str.replace('$','')
# Convert the column values from object to float64
df_all['Valuation ($B)'] = pd.to_numeric(df_all['Valuation ($B)'])
# Convert the 'Date Joined' column values into datatime64[ns] format
df_all['Date Joined'] = pd.to_datetime(df_all['Date Joined'])
# Sort the companies in descending order of valuation
df_all.sort_values(['Valuation ($B)'], ascending=[0])
df_all.head()

C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_49568\3890740901.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_all['Valuation ($B)'] = df_all['Valuation ($B)'].str.replace('$','')


,Unnamed: 0,Company,Valuation ($B),Date Joined,Country,City,Industry,Select Investors
0,0,Bytedance,140.0,2017-04-07,China,Beijing,Artificial intelligence,"Sequoia Capital China, SIG Asia Investments, S..."
1,1,SpaceX,125.0,2012-12-01,United States,Hawthorne,Other,"Founders Fund, Draper Fisher Jurvetson, Rothen..."
2,2,SHEIN,100.0,2018-07-03,China,Shenzhen,E-commerce & direct-to-consumer,"Tiger Global Management, Sequoia Capital China..."
3,3,Stripe,95.0,2014-01-23,United States,San Francisco,Fintech,"Khosla Ventures, LowercaseCapital, capitalG"
4,4,Klarna,45.6,2011-12-12,Sweden,Stockholm,Fintech,"Institutional Venture Partners, Sequoia Capita..."


#### Use bokeh bar plot to plot all the unicorn companies valuation.
Target : Visualize the vast diffrences in funding amount

In [53]:
# Create a new plot with a title and axis labels
valuation_biggest_companies = figure(title="Valuation", x_axis_label="COMPANY NAME", y_axis_label="VALUATION")

# Add bar graph renderer
valuation_biggest_companies.vbar(x=df_all['Unnamed: 0'], top=df_all['Valuation ($B)'], legend_label="USD $B", width=0.5, bottom=0, color="blue")
# valuation_biggest_companies.circle(x, y3, legend_label="Objects", color="yellow", size=12)

# Display the results
show(valuation_biggest_companies)